In [ ]:
# default_exp cli

In [ ]:
#export
import argparse, os, sys
import fire
from fire import core
from Bio import SeqIO
import pandas as pd
from pathlib import Path

In [ ]:
#export
from duomolog import blast
from duomolog import msa
from duomolog import hmmer

In [ ]:
#export
def Display(lines, out):
    text = "\n".join(lines) + "\n"
    out.write(text)
def main():
    commands = CLI()
    core.Display = Display
    fire.Fire(commands, name="duomolog")
    # print(say_hello("Jeremy")=="Hello Jeremy!")

# Duomolog command line options

First attempt.

This is what I have in the original.

In [ ]:
#hide
test = CLI()

In [ ]:
#hide
test.blast_v_hmmer(inFile="data/enolase.part001.fa",queryFile="data/enolase.part002.fa" ,summaryOut="duomolog_out_00/summary_out.txt")

In [ ]:
test.blastOut

In [ ]:
#export
class CLI(object):
    def __init__(self):
        self.blastOut = None
        self.hmmerOut = None
    def blast_v_hmmer(self, 
		inFile, 
		queryFile,
		summaryOut="duomolog_out/summary_out.txt",
		blastFile=None,
        alnFile=None,
		hmmFile=None,
        intersectOnly=False):

        os.makedirs(os.path.dirname(summaryOut), exist_ok=True)
        Path(summaryOut).touch() # should at worst guarantee output is generated
        outDir = '/'.join(summaryOut.split("/")[:-1]) +"/"
        in_nrPepDB = nrPepDB(inFile) 
        in_nrPepDB.to_csv(inFile+".nr.csv")
        in_nrFile = outDir + "in_nr.fasta"
        pepDB2Fa(in_nrPepDB,in_nrFile)

        query_nrPepDB = nrPepDB(queryFile) 
        query_nrPepDB.to_csv(queryFile+".nr.csv")
        query_novelDB, perfectMatchDB = perfectMatch(in_nrPepDB,query_nrPepDB)
        query_novel_file = outDir + "query_novel.fasta"
        perfectMatch_FileHandle = outDir + "perfectMatch"
        
        if not perfectMatchDB.empty:
            pepDB2Fa(perfectMatchDB,f"{perfectMatch_FileHandle}.fasta",      
            recordCol="record_y")
            with open(f"{perfectMatch_FileHandle}.txt","w") as out:
                out.write("queryID\tmatchID\n")
                for index, row in perfectMatchDB.iterrows():
                    for queryID in row['allHeaders_x'].split(";"):
                        for matchID in row['allHeaders_y'].split(";"):
                            out.write(f"{queryID}\t{matchID}\n")
            
        pepDB2Fa(query_novelDB,query_novel_file)
        
        if blastFile == None:
            self.blastOut = blast.run_blast(in_nrFile, query_novel_file)
        else:
            inHeaders = in_nrPepDB["1stHeader"]
            self.blastOut = blast.load_blast(blastFile,inHeaders)
        
        if hmmFile == None:
            alnFile, alnOut = msa.run_mafft(in_nrFile)
        self.hmmerOut = hmmer.run_hmmsearch(query_novel_file, alnFile, hmmFile)

        blast_headers = set(self.blastOut["qseqid"])
        hmmer_headers = set([hit.name.decode() for hit in self.hmmerOut])
        blast_hmmer_duo = Duo("blast",blast_headers,"hmmer",hmmer_headers)
        
        self.blastOut.to_csv(f"{outDir}blastout.txt",sep='\t',index=False)
        writeOut(outDir, summaryOut, blast_hmmer_duo, query_novelDB, intersectOnly)


In [ ]:
#export
class Duo:
	def __init__(self, leftName, left, rightName, right):
		self.left_union_rightName = leftName + "_union_" + rightName
		self.left_not_rightName = leftName + "_not_" + rightName
		self.left_intersect_rightName = leftName + "_intersect_" + rightName
		self.right_not_leftName = rightName + "_not_" + leftName

		self.subsets = {
			leftName: left,
			rightName: right,
			self.left_union_rightName: left | right,
			self.left_not_rightName: left - right,
			self.left_intersect_rightName: left & right,
			self.right_not_leftName: right - left
		}

	def dropEmpty(self):
		emptySubsets = []
		for subset in self.subsets:
			if self.subsets[subset] == set():
				print(subset, "is empty, removing it")
				emptySubsets.append(subset)
		for emptySubset in emptySubsets: 
			del self.subsets[emptySubset]

	def dropRedundant(self):
		nrSubSets = []
		redundantSubsets = []
		for subset in self.subsets:
			if self.subsets[subset] not in nrSubSets:
				nrSubSets.append(self.subsets[subset])
			else:
				print(subset, "is redundant, removing it")
				redundantSubsets.append(subset)
		for redundantSubset in redundantSubsets:
			del self.subsets[redundantSubset]
	def intersectOnly(self):
		self.subsets = {self.left_intersect_rightName:self.subsets[self.left_intersect_rightName]}


In [ ]:
#export
def writeOut(outDir,summaryOut,duo,query_novelDB, intersectOnly):
    if intersectOnly:
        duo.intersectOnly()
    else:
        duo.dropRedundant()
    duo.dropEmpty()
    if bool(duo):
        with open(summaryOut, "w") as out:
            for subset in duo.subsets:
                subsetDB = subPepDB(query_novelDB, duo.subsets[subset])
                pepDB2Fa(subsetDB, outDir + f"{subset}.fa")
                for header in duo.subsets[subset]:
                    out.write(f"{header}\t{subset}")


In [ ]:
#export
def nrPepDB(faFile):
    seqDict = {}
    outDict = {"1stHeader":[],  "allHeaders":[],"record":[],"seq":[]}
    with open(faFile) as fa:
        for record in SeqIO.parse(fa, "fasta"):
            if record.seq not in seqDict:
                seqDict[record.seq] = [record]
            else:
                print(f"WARNING: {record.id} contains a redundant sequence")
                seqDict[record.seq].append(record)
    
    for seq in seqDict:
        h1 = seqDict[seq][0].id
        # allHs = ';'.join(seqDict[seq])
        allHs = ';'.join([r.id for r in seqDict[seq]])
        outDict["1stHeader"].append(h1)
        outDict["allHeaders"].append(allHs)
        outDict["record"].append(seqDict[seq][0])
        outDict["seq"].append(str(seq))

    
    outDB = pd.DataFrame.from_dict(outDict)
    outDB["1stHeader"].astype(str)
    return outDB





In [ ]:
#hide
test_nrDB = nrPepDB("data/redundantSeqs.fa")

In [ ]:
#export
def pepDB2Fa(pepDB, faFile,recordCol="record",format="fasta"):
    SeqIO.write(pepDB[recordCol], faFile,format)


In [ ]:
#export
def subPepDB(pepDB, headers):
    return pepDB[pepDB["1stHeader"].isin(headers)]


In [ ]:
#export
def perfectMatch(inDB,queryDB):
    inSeqs = set(inDB["seq"])
    querySeqs = set(queryDB["seq"])
    # print(inSeqs)
    # print(querySeqs)
    sharedSeqs = inSeqs & querySeqs
    sharedDB = queryDB[queryDB["seq"].isin(sharedSeqs)]
    inner_mergeDB = inner_mergePD(inDB,sharedDB,"seq")
    novelDB = queryDB[~queryDB["seq"].isin(sharedSeqs)]
    


    return novelDB, inner_mergeDB


In [ ]:
#export
def inner_mergePD(x,y,commonCol):
    return x.merge(y, on = commonCol)

In [ ]:
#export
def getOtherColumnValue(df,c1,c2,c1_value):
    # return df[df[c1] == c1_value][c2][0]
    return df.loc[df[c1] == c1_value, c2].iloc[0]


In [ ]:
#hide
getOtherColumnValue(p1_nrDB,"1stHeader","seq","4416387")

In [ ]:
#hide
p1_nrDB = nrPepDB("data/enolase.part001.fa")
p2_nrDB = nrPepDB("data/enolase.part002.fa")

p2_novelDB, p2_shared_with_p1 = perfectMatch(p1_nrDB,p2_nrDB)

p2_shared_with_p1

In [ ]:
#hide
not p2_shared_with_p1.empty

In [ ]:
#hide
for index, row in p2_insert_seqFrom_p1_shared.iterrows():
    for queryID in row['allHeaders_x'].split(";"):
        for matchID in row['allHeaders_y'].split(";"):
            print(queryID,matchID)

In [ ]:
p2_insert_seqFrom_p1DB = nrPepDB("data/enolase.part002.insert_seqFrom_part1.fa")
p2_insert_seqFrom_p1_novel , p2_insert_seqFrom_p1_shared = perfectMatch(p1_nrDB,p2_insert_seqFrom_p1DB)
p2_insert_seqFrom_p1_shared

In [ ]:
p2_insert_seqFrom_p1_shared

So here we see that 17137654_copy is shared with p1_nrDB, we need to get the header that it matched with

In [ ]:
inner_mergePD(p1_nrDB,p2_insert_seqFrom_p1_shared,"seq")

In [ ]:
p1_nrDB.merge(p2_insert_seqFrom_p1_shared, on = "seq")
# "17137654_copy"

In [ ]:
#hide
p2_novelDB

In [ ]:
#hide
test_inDB = pd.DataFrame.from_dict({"seq":[1,2,3],"h":[1,2,3]})
test_queryDB = pd.DataFrame.from_dict({"seq":[2,4,6],"h":[1,2,3]})
perfectMatch(test_inDB,test_queryDB)



In [ ]:
#hide
from nbdev.showdoc import *